# Import libraries and load functions

In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import DescrStatsW
from functions_metabodirect import *

currentDT = datetime.datetime.now()
print(currentDT.strftime("%Y-%m-%d %H:%M:%S %p"))
print(os.getcwd())

2021-02-03 11:43:32 AM
/Volumes/NGG_TFAILY_LAB_1/Fusarium_wilt_Lettuce/direct_injection/Water_fraction


# Make project directory

In [2]:
project_dir = os.getcwd()
project_name = 'MetaboDirect'
preprocess_dir = '1_preprocessing_output'
diagnostics_dir = '2_diagnostics'
exploratory_dir = '3_exploratory'
stats_dir = '4_statistics'
transf_dir = '5_transformations'

list_dir = [
    preprocess_dir, diagnostics_dir, exploratory_dir, stats_dir, transf_dir
]

list_dir = [os.path.join(project_name, x) for x in list_dir]

# assert os.path.exists(project_name) == False

if not os.path.exists(project_name):
    os.makedirs(project_name)

for d in list_dir:
    if not os.path.exists(d):
        os.makedirs(d)

# Preprocessing

In the preprocessing, we'll take the output from Formularity, a file called `Report.csv`. It's important that you work with the output file from Formularity, and do not modify that file in any way. 

These are the steps in the preprocessing:

1. Filter predicted formulas with 13C
2. Optional: filter m/z between a m/z range
3. Calculate soil organic C indices and class of compounds
5. Calculate a summary output of elemental and class composition per sample
5. Calculate a summary output of median and weighted mean of indices per sample
6. Make matrix for downstream analyses

In [3]:
df = pd.read_csv('Report.csv')
print(f"Number of m/z: {df.shape[0]}")

Number of m/z: 356751


## Filter predicted formulas with 13C

In [4]:
filt_df, n_excluded = filter_C13(df)

print(f"Number of excluded m/z: {n_excluded}")
print(f"Number of m/z: {filt_df.shape[0]}")

Number of excluded m/z: 39789
Number of m/z: 316962


## Optional: filter m/z between a m/z range

In [5]:
to_filter = True
min_mz, max_mz = 200, 900 # default values
filt_df = filter_mz(filt_df, to_filter, min_mz, max_mz) 

print(f"Number of m/z: {filt_df.shape[0]}")

Number of m/z: 250300


## Filter error (ppm) range

In [6]:
filt_df = filter_error_ppm(filt_df, err_range=0.5)

print(f"Number of m/z: {filt_df.shape[0]}")

Number of m/z: 230154


## Calculate soil organic C indices and class of compounds

Soil organic C indices include: O:C, H:C, NOSC, GFE, DBE, DBE modified, AI, AI modified, and compound classes based on ratios and boundaries by M. Tfaily (we don't consider the calculations by Formularity).

These are the boundaries by M. Tfaily:

| Class          | O:C (low) | O:C (high)| H:C (low)| H:C (high)|
| :------- | :------- | :------- | :------- | :------- |
| Lipid          |    >0     |     0.3   |    1.5   |     2.5  | 
| Unsaturated HC |    0      |     0.125 |    0.8   |    <1.5  |
| Condensed HC   |    0      |     0.95  |    0.2   |    <0.8  |
| Protein        |    >0.3   |     0.55  |    1.5   |     2.3  |
| Amino sugar    |    >0.55  |     0.7   |    1.5   |     2.2  |
| Carbohydrate   |    >0.7   |     1.5   |    1.5   |     2.5  |
| Lignin         |    >0.125 |     0.65  |    0.8   |    <1.5  |
| Tannin         |    >0.65  |     1.1   |    0.8   |    <1.5  |

In [7]:
final_df = calculate_ratios(filt_df)
final_df = calculate_classes(final_df)
final_df = normalize_intensities(final_df)

In [8]:
# save preprocessed df
filename = os.path.join(project_name, preprocess_dir, 'Report_processed.csv')
final_df.to_csv(filename, index=False)

# save preprocessed df (removing m/z without a molecular formula)
final_df_formulas = final_df[final_df['C']>0]
filename = os.path.join(project_name, preprocess_dir, 'Report_processed_MolecFormulas.csv')
final_df_formulas.to_csv(filename, index=False)

## Calculate a summary output with the percent classes per sample

In [9]:
class_comp = get_summary(final_df_formulas, on='Class')
el_comp = get_summary(final_df_formulas, on='El_comp')

# save class composition
filename = os.path.join(project_name, preprocess_dir, 'class_composition.csv')
class_comp.to_csv(filename, index=False)

# save elemental composition
filename = os.path.join(project_name, preprocess_dir, 'elemental_composition.csv')
el_comp.to_csv(filename, index=False)

## Calculate a summary output of median and weighted mean of indices per sample

In [10]:
nosc = get_summary_indices(final_df_formulas, 'NOSC')
gfe = get_summary_indices(final_df_formulas, 'GFE')
# can do the same for AI, DBE, etc.

idx_stats = nosc.merge(gfe, on='SampleID')

# save
filename = os.path.join(project_name, preprocess_dir, 'indices_statistics.csv')
idx_stats.to_csv(filename, index=False)

## Make matrix for downstream analyses

In [11]:
# matrix = get_matrix(final_df)
matrix_formulas = get_matrix(final_df_formulas)

# save matrix 
filename = os.path.join(project_name, preprocess_dir, 'matrix_features.csv')
matrix_formulas.to_csv(filename)

# Metadata

I had to exclude a few samples because they didn't calibrate in Formularity, so I'll just make the metadata from the sample names.

In [12]:
metadata = pd.DataFrame()

samples = get_list_samples(final_df_formulas)
metadata['SampleID'] = samples

metadata[['Cultivar', 'Time', 'Treatment', 'Tissue', 'Rep']] = ''

metadata['Cultivar'] = metadata['SampleID'].str.split('_').str[0]
metadata['Cultivar'] = metadata['Cultivar'].replace(['R', 'S', 'Foc'], ['Tolerant', 'Susceptible', 'Fungus'])

metadata['Time'] = metadata['SampleID'].str.split('_').str[1]
metadata['Time'] = metadata['Time'].replace(['W1', 'W15', 'W2'], ['7 dpi', '10 dpi', '14 dpi'])

metadata['Treatment'] = metadata['SampleID'].str.split('_').str[2]
metadata['Treatment'] = metadata['Treatment'].replace(['I', 'NI'], ['inoculated', 'healthy'])

metadata['Tissue'] = metadata['SampleID'].str.split('_').str[3].str[0]
metadata['Tissue'] = metadata['Tissue'].replace(['L', 'R'], ['leaf', 'root'])

metadata['Rep'] = metadata['SampleID'].str.split('_').str[3].str[-1]

metadata.loc[metadata['SampleID'].str.startswith('Foc'), 'Time'] = '7 dpi'
metadata.loc[metadata['SampleID'].str.startswith('Foc'), 'Treatment'] = 'Fungus'
metadata.loc[metadata['SampleID'].str.startswith('Foc'), 'Tissue'] = 'Fungus'

for i in [0,1,2,3]:
    metadata.loc[i, 'Rep'] = i+1
    
metadata['Names'] = metadata['Cultivar']+' '+metadata['Tissue']+' '+metadata['Treatment']+' '+metadata['Time']

for i in [0,1,2,3]:
    metadata.loc[i, 'Names'] = 'Fungus 7 dpi'

metadata.to_csv('metadata.csv', index=False)